<a href="https://colab.research.google.com/github/chandradutta/DL_Assignment_2/blob/main/Assignmnet2_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torch.nn.functional as func
# import wandb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchvision
import torchvision


In [ ]:
!ls

In [ ]:
dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/inaturalist_12K', transform=transforms)

In [ ]:
from torchvision import transforms
transform = transforms.Compose([transforms.Resize((200,200)),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229, 0.224, 0.225])])

In [ ]:

train_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train', transform=transform)
test_dataset = ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/val', transform=transform)


# Define the size of the validation set (20%)
validation_size = 0.2

# Split the dataset into training and validation sets with shuffling
train_data, val_data = train_test_split(list(range(len(train_dataset))), test_size=validation_size, shuffle=True, random_state=42)

#Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# dataiter = iter(train_loader)
# data = next(dataiter)
# inputs, targets = data
# print(inputs.shape, targets.shape)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))


In [ ]:
print(images.shape)

In [ ]:
import math
def compute_size(input_size, kernel_size,stride):
      padding=0
      output_size = math.floor((input_size - kernel_size + 2 * padding) / stride) + 1
      return output_size
class ConvNetwork(nn.Module):
  def __init__(self,n_filters,filter_size,activationFun,denseLay):
        super(ConvNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3,n_filters,filter_size,stride=1)
        self.size_after_conv1 = compute_size(200, filter_size, stride=1)
        self.pool1 = nn.MaxPool2d(filter_size,stride=2)
        self.size_after_pool1 = compute_size(self.size_after_conv1, filter_size, stride=2)

        self.conv2 = nn.Conv2d(n_filters, n_filters*2,filter_size,stride=1)
        self.size_after_conv2 = compute_size(self.size_after_pool1, filter_size, stride=1)
        self.pool2 = nn.MaxPool2d(filter_size,stride=2)
        self.size_after_pool2 = compute_size(self.size_after_conv2, filter_size, stride=2)

        self.conv3 = nn.Conv2d(n_filters*2,n_filters*4,filter_size,stride=1)
        self.size_after_conv3 = compute_size(self.size_after_pool2, filter_size, stride=1)
        self.pool3= nn.MaxPool2d(filter_size,stride=2)
        self.size_after_pool3 = compute_size(self.size_after_conv3, filter_size, stride=2)

        self.conv4 = nn.Conv2d(n_filters*4,n_filters*8,filter_size,stride=1)
        self.size_after_conv4 = compute_size(self.size_after_pool3, filter_size, stride=1)
        self.pool4 = nn.MaxPool2d(filter_size,stride=2)
        self.size_after_pool4 = compute_size(self.size_after_conv4, filter_size, stride=2)


        self.conv5 = nn.Conv2d(n_filters*8,n_filters*16,filter_size,stride=1)
        self.size_after_conv5 = compute_size(self.size_after_pool4, filter_size, stride=1)
        self.pool5 = nn.MaxPool2d(filter_size,stride=2)
        self.size_after_pool5 = compute_size(self.size_after_conv5, filter_size, stride=2)


        # Fully connected layers
        self.fc1 = nn.Linear(n_filters*16 * (self.size_after_pool5) **2, denseLay)
        self.opLay = nn.Linear(denseLay, 10)

  # def compute_size(input_size, kernel_size,stride):
  #     padding=0
  #     output_size = (input_size - kernel_size + 2 * padding) / stride + 1
  #     return output_size

  def forward(self, x):
        x = self.pool1(func.relu(self.conv1(x)))
        x = self.pool2(func.relu(self.conv2(x)))
        x = self.pool3(func.relu(self.conv3(x)))
        x = self.pool4(func.relu(self.conv4(x)))
        x = self.pool5(func.relu(self.conv5(x)))
        x=x.reshape(x.shape[0],-1)

        x = func.relu(self.fc1(x))
        x = self.opLay(x)
        return x

learning_rate=0.001
epochs=5
n_filters = 32
filter_size = 3
denseLay = 128
activationFun='relu'

Network=ConvNetwork(n_filters,filter_size,activationFun,denseLay)

LossFun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(Network.parameters(), lr=learning_rate)

length_of_data = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):

        # Forward pass
        outputs = Network(images)
        loss = LossFun(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{length_of_data}], Loss: {loss.item():.4f}')

In [ ]:
import torch